In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
from pathlib import Path

import numpy as np
import pandas as pd

# Load data

In [ ]:
braf_kras = pd.read_csv('braf_kras_experiments.csv', delimiter=",")

In [ ]:
braf_kras.Train.unique().tolist()

In [ ]:
braf_kras = braf_kras.drop(columns=['Number', 'Experiment', 'version', 'Normalization', 'Feature Extraction',
        'Comments', 'auroc std', 'auprc mean',
        'auprc std', 'f1_50 mean', 'f1_50 std', 'f1_spec90 mean',
        'f1_spec90 std', 'f1_spec95 mean', 'f1_spec95 std', 'f1_gmean mean',
        'f1_gmean std', 'f1_jstat mean', 'f1_jstat std'])
braf_kras = braf_kras[braf_kras["Train"] != 'Dachs, Quasar, Rainbow, TCGA']

In [ ]:
braf_kras_histaugan = pd.read_csv('braf_kras_histaugan.csv', delimiter=",")

In [ ]:
braf_kras_histaugan = braf_kras_histaugan.drop(columns=['Number', 'Experiment', 'version', 'Normalization', 'Feature Extraction',
        'Comments', 'auroc std', 'auprc mean',
        'auprc std', 'f1_50 mean', 'f1_50 std', 'f1_spec90 mean',
        'f1_spec90 std', 'f1_spec95 mean', 'f1_spec95 std', 'f1_gmean mean',
        'f1_gmean std', 'f1_jstat mean', 'f1_jstat std'])
braf_kras_histaugan = braf_kras_histaugan[braf_kras_histaugan["Train"] != 'Dachs, Quasar, Rainbow, TCGA']

In [ ]:
braf_kras;

In [ ]:
echles = [
    [0.74, 0.76, 0.77, 0.72],
    [0.67, 0.89, 0.71, 0.76],
    [0.81, 0.68, 0.92, 0.80],
    [0.77, 0.80, 0.82, 0.90],
]
echles = np.array(echles)

# heatmaps with dataframes

In [ ]:
figure_path = Path('/Users/sophia.wagner/Documents/PhD/projects/2022_MSI_transformer/figures')

In [ ]:
norm = 'histaugan'
target = 'isMSIH'
sheet_name = '1-ctp-cls-histaugan-idkidc'
# sheet_name = '3-braf-histaugan-idkidc'
# sheet_name = '3-kras-histaugan-idkidc'

# results = Path('/Users/sophia.wagner/Documents/PhD/projects/idkidc/MSI Transformer Experiments - 1-ctp-cls-histaugan-idkidc.csv')
# results = pd.read_csv(results, delimiter=",")

results = Path('/Users/sophia.wagner/Documents/PhD/projects/idkidc/MSI Transformer Experiments.xlsx')
results = pd.read_excel(results, sheet_name=sheet_name)

cohorts = ['CPTAC', 'DACHS', 'DUSSEL', 'Epi700', 'ERLANGEN', 'FOXTROT', 'MCO', 'MECC', 'MUNICH', 'QUASAR', 'RAINBOW', 'TCGA', 'TRANSCOT', 'YCR-BCIP-resections']
# cohorts = ['DACHS', 'Epi700', 'MCO', 'QUASAR', 'RAINBOW', 'TCGA']
# cohorts = results['Test'].unique().tolist()
print(cohorts)

In [ ]:
for k in results.keys():
    if k not in ['Train', 'Test', 'auroc/test mean']:
        results = results.drop(columns=[k])

In [ ]:
results

In [ ]:
cohort_size = {
    'CHINA': 35,
    'CPTAC': 105,
    'DACHS': 2039,
    'DUSSEL': 196,
    'Epi700': 603, 
    'ERLANGEN': 458,
    'FOXTROT': 702,
    'MAINZ': 86,
    'MCO': 1388, 
    'MECC': 683, 
    'MUNICH': 287, 
    'QUASAR': 1774,
    'RAINBOW': 2068, 
    'TCGA': 426, 
    'TRANSCOT': 1972, 
    'YCR-BCIP-resections': 867
}

cohort_label = {
    'CHINA': 'GUANGZHOU',
    'CPTAC': 'CPTAC',
    'DACHS': 'DACHS',
    'DUSSEL': 'DUSSEL',
    'Epi700': 'Epi700', 
    'ERLANGEN': 'ERLANGEN',
    'FOXTROT': 'FOXTROT',
    'MAINZ': 'MAINZ',
    'MCO': 'MCO', 
    'MECC': 'MECC', 
    'MUNICH': 'MUNICH', 
    'QUASAR': 'QUASAR',
    'RAINBOW': 'NLCS', 
    'TCGA': 'TCGA', 
    'TRANSCOT': 'TRANSCOT', 
    'YCR-BCIP-resections': 'YCR-BCIP',
}

In [ ]:
# sort cohorts by size
cohorts_ordered = sorted(cohorts, key=lambda x: cohort_size[x], reverse=True)
size_ordered = [cohort_size[c] for c in cohorts_ordered]
label_ordered = [cohort_label[c] for c in cohorts_ordered]

In [ ]:
heatmap = np.zeros((len(cohorts_ordered), len(cohorts_ordered)))
for i, c_i in enumerate(cohorts_ordered):
    for j, c_j in enumerate(cohorts_ordered):
        try: 
            heatmap[j, i] = results[results['Train'] == c_i][results['Test'] == c_j]['auroc/test mean'].values[0]
        except IndexError:
            continue

In [ ]:
# name = 'histaugan'
name = sheet_name
labels = False

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=((6/5)*5, 5), gridspec_kw={'width_ratios': [5, 1], 'wspace':0, 'hspace':0}) # figsize=(7, 5) 'width_ratios': [6, 1]
im = ax[0].imshow(heatmap.T, vmin=0.55, vmax=1, cmap='plasma')

if labels:
    # Loop over data dimensions and create text annotations.
    for i in range(len(cohorts_ordered)):
        for j in range(len(cohorts_ordered)):
            text = ax[0].text(j, i, f'{heatmap.T[i][j]:.2f}',
                        ha="center", va="center", color="w", fontsize=14)

ax[0].set_xticks(np.arange(len(label_ordered)), labels=label_ordered)
ax[0].set_yticks(np.arange(len(label_ordered)), labels=label_ordered)
# ax[0].set_yticks(np.arange(len(cohorts)), labels=cohorts)

plt.setp(ax[0].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

fig.tight_layout()

ax[0].set(
    title=f"AUROC scores (target: {target})",
)

ax[0].set_ylabel('Train')
ax[0].set_xlabel('Test')

# plot bar plot
# ax[1].set_box_aspect(5/1)
ax[1].barh(range(len(size_ordered), 0, -1), size_ordered, color='slategray')

ax[1].axes.get_yaxis().set_visible(False)
ax[1].set_xticks((0, 1000, 2000))
plt.setp(ax[1].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
# ax[1].spines['bottom'].set_visible(False)
# ax[1].spines['left'].set_visible(False)
ax[1].set(
    ylim=[0.5, len(size_ordered) + 0.5],
    # ylim=[-0.05, 1.05],
)
ax[1].set_xlabel('Counts')

fig.savefig(figure_path / f'results_heatmap_{name}_new_cohorts.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
cohorts = ['RAINBOW', 'DACHS', 'QUASAR', 'TCGA']

# sort cohorts by size
cohorts_ordered = sorted(cohorts, key=lambda x: cohort_size[x], reverse=True)
size_ordered = [cohort_size[c] for c in cohorts_ordered]
label_ordered = [cohort_label[c] for c in cohorts_ordered]

In [ ]:
%%capture
heatmap = np.zeros((len(cohorts_ordered), len(cohorts_ordered)))
for i, c_i in enumerate(cohorts_ordered):
    for j, c_j in enumerate(cohorts_ordered):
        try: 
            heatmap[j, i] = results[results['Train'] == c_i][results['Test'] == c_j]['auroc/test mean'].values[0]
        except IndexError:
            continue

In [ ]:
name = 'histaugan'

fig, ax = plt.subplots(figsize=(1.875, 1.875))
# im = ax.imshow(np.flip(ours[:4, :4].T), vmin=0.55, vmax=1, cmap='plasma')
im = ax.imshow(heatmap.T, vmin=0.55, vmax=1, cmap='plasma')


ax.set_xticks(np.arange(len(cohorts)), labels=label_ordered)
ax.axes.get_xaxis().set_visible(False)

ax.set_yticks(np.arange(len(cohorts)), labels=label_ordered)

labels = heatmap.T
# Loop over data dimensions and create text annotations.
for i in range(len(cohorts)):
    for j in range(len(cohorts)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=10)
        
# ax.set(
#     title="Transformer",
# )

# ax.set_xlabel('Train')
ax.set_ylabel('Transformer')

fig.savefig(figure_path / f'results_transformer_{name}_zoom.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()

## heatmaps for MSI (Ours)

In [ ]:
cohorts = ['TCGA', 'QUASAR', 'DACHS', 'NLCS', 'Epi700', 'CPTAC', 'DUSSEL', 'MECC', 'MUNICH', 'YCR-BCIP']
data_size = [426, 1774, 2039,2068, 603, 105, 196, 683, 287, 867]

In [ ]:
plt.bar(range(len(data_size)), data_size)

In [ ]:
# ours = [
#     [0.86, 0.87, 0.87, 0.85, 0.83, 0.75, 0.68, 0.80, 0.81, 0.86, 0.87],
#     [0.87, 0.92, 0.92, 0.92, 0.91, 0.75, 0.77, 0.84, 0.84, 0.86, 0.92],
#     [0.86, 0.89, 0.96, 0.89, 0.88, 0.75, 0.73, 0.83, 0.82, 0.86, 0.89],
#     [0.85, 0.91, 0.92, 0.93, 0.91, 0.78, 0.72, 0.84, 0.83, 0.88, 0.91],
#     [0.85, 0.91, 0.92, 0.90, 0.92, 0.81, 0.74, 0.81, 0.84, 0.89, 0.91],
#     [0.82, 0.89, 0.85, 0.90, 0.84, 0.86, 0.67, 0.72, 0.71, 0.88, 0.91],
#     [0.76, 0.76, 0.81, 0.76, 0.75, 0.72, 0.75, 0.75, 0.79, 0.78, 0.79],
#     [0.75, 0.74, 0.75, 0.75, 0.75, 0.67, 0.66, 0.73, 0.85, 0.73, 0.73],
#     [0.86, 0.85, 0.86, 0.81, 0.88, 0.73, 0.71, 0.79, 0.73, 0.80, 0.85],
#     [0.80, 0.84, 0.87, 0.86, 0.84, 0.73, 0.68, 0.62, 0.69, 0.94, 0.88],
#     [0.91, 0.94, 0.93, 0.94, 0.92, 0.83, 0.82, 0.86, 0.87, 0.92, 0.95],
# ]
# ours = np.array(ours)

# data with belfast as Epi700 in alphabetic order
ours = [
    [0.86, 0.87, 0.87, 0.85, 0.74, 0.72, 0.85, 0.79, 0.78, 0.87],
    [0.84, 0.94, 0.91, 0.91, 0.72, 0.79, 0.90, 0.81, 0.73, 0.92],
    [0.85, 0.88, 0.95, 0.88, 0.73, 0.77, 0.87, 0.84, 0.77, 0.89],
    [0.83, 0.91, 0.90, 0.93, 0.77, 0.74, 0.92, 0.84, 0.78, 0.90],
    [0.84, 0.89, 0.83, 0.90, 0.86, 0.70, 0.85, 0.71, 0.64, 0.91],
    [0.71, 0.75, 0.81, 0.74, 0.71, 0.68, 0.76, 0.72, 0.74, 0.81],
    [0.85, 0.89, 0.90, 0.90, 0.81, 0.75, 0.89, 0.82, 0.82, 0.91],
    [0.73, 0.75, 0.74, 0.74, 0.67, 0.68, 0.74, 0.72, 0.68, 0.74],
    [0.80, 0.85, 0.86, 0.81, 0.70, 0.74, 0.88, 0.77, 0.79, 0.86],
    [0.88, 0.95, 0.93, 0.94, 0.83, 0.83, 0.93, 0.84, 0.80, 0.95],
]
ours = np.array(ours)
cohorts = ['TCGA', 'QUASAR', 'DACHS', 'NLCS', 'CPTAC', 'DUSSEL', 'Epi700', 'MECC', 'MUNICH', 'YCR-BCIP']
data_size = [426, 1774, 2039,2068, 105, 196, 603, 683, 287, 867]

In [ ]:
# res_only = np.delete(np.delete(ours, 9, axis=0), 9, axis=1)
# print(res_only)

In [ ]:
# completely alphabetically sorted
ours = [
    [0.86, 0.83, 0.70, 0.85, 0.71, 0.64, 0.90, 0.89, 0.84, 0.91],
    [0.73, 0.95, 0.77, 0.87, 0.84, 0.77, 0.88, 0.88, 0.85, 0.89],
    [0.71, 0.81, 0.68, 0.76, 0.72, 0.74, 0.74, 0.75, 0.71, 0.81],
    [0.81, 0.90, 0.75, 0.89, 0.82, 0.82, 0.90, 0.89, 0.85, 0.91],
    [0.67, 0.74, 0.68, 0.74, 0.72, 0.68, 0.74, 0.75, 0.73, 0.74],
    [0.70, 0.86, 0.74, 0.88, 0.77, 0.79, 0.81, 0.85, 0.80, 0.86],
    [0.77, 0.90, 0.74, 0.92, 0.84, 0.78, 0.93, 0.91, 0.83, 0.90],
    [0.72, 0.91, 0.79, 0.90, 0.81, 0.73, 0.91, 0.94, 0.84, 0.92],
    [0.74, 0.87, 0.72, 0.85, 0.79, 0.78, 0.85, 0.87, 0.86, 0.87],
    [0.83, 0.93, 0.83, 0.93, 0.84, 0.80, 0.94, 0.95, 0.88, 0.95],
]
ours = np.array(ours)
cohorts = [ 'CPTAC',    'DACHS',    'DUSSEL',   'Epi700',   'MECC',     'MUNICH',   'NLCS',     'QUASAR',   'TCGA',     'YCR-BCIP']
data_size = [105,       2039,       196,        603,        683,        287,        2068,       1774,       426,        867]

In [ ]:
# sorted by size of the dataset (32 epochs)
# macenko = [
#     [0.93, 0.90, 0.91, 0.90, 0.84, 0.92, 0.83, 0.78, 0.74, 0.77],
#     [0.88, 0.95, 0.88, 0.89, 0.84, 0.87, 0.85, 0.77, 0.77, 0.73],
#     [0.91, 0.91, 0.94, 0.92, 0.81, 0.90, 0.84, 0.73, 0.79, 0.72],
#     [0.94, 0.93, 0.95, 0.95, 0.84, 0.93, 0.88, 0.80, 0.83, 0.83],
#     [0.74, 0.74, 0.75, 0.74, 0.72, 0.74, 0.73, 0.68, 0.68, 0.67],
#     [0.90, 0.90, 0.89, 0.91, 0.82, 0.89, 0.85, 0.82, 0.75, 0.81],
#     [0.85, 0.87, 0.87, 0.87, 0.79, 0.85, 0.86, 0.78, 0.72, 0.74],
#     [0.81, 0.86, 0.85, 0.86, 0.77, 0.88, 0.80, 0.79, 0.74, 0.70],
#     [0.74, 0.81, 0.75, 0.81, 0.72, 0.76, 0.71, 0.74, 0.68, 0.71],
#     [0.90, 0.83, 0.89, 0.91, 0.71, 0.85, 0.84, 0.64, 0.70, 0.86],
# ]
# sorted by size of the dataset (8 epochs)
macenko = [
    [0.93, 0.92, 0.92, 0.91, 0.84, 0.90, 0.83, 0.86, 0.76, 0.78],
    [0.92, 0.96, 0.92, 0.89, 0.82, 0.87, 0.84, 0.86, 0.79, 0.76],
    [0.93, 0.92, 0.95, 0.92, 0.81, 0.88, 0.85, 0.87, 0.79, 0.75],
    [0.96, 0.93, 0.95, 0.96, 0.88, 0.92, 0.89, 0.87, 0.85, 0.83],
    [0.79, 0.76, 0.76, 0.76, 0.73, 0.76, 0.75, 0.74, 0.67, 0.68],
    [0.93, 0.91, 0.93, 0.91, 0.82, 0.90, 0.84, 0.85, 0.75, 0.88],
    [0.88, 0.87, 0.88, 0.88, 0.78, 0.85, 0.86, 0.82, 0.73, 0.74],
    [0.86, 0.84, 0.88, 0.87, 0.73, 0.84, 0.82, 0.85, 0.74, 0.70],
    [0.80, 0.79, 0.81, 0.83, 0.79, 0.76, 0.75, 0.78, 0.78, 0.73],
    [0.90, 0.88, 0.91, 0.91, 0.80, 0.85, 0.87, 0.75, 0.73, 0.81],   
]

# sorted by size o
macenko = np.array(macenko)
cohorts = ['NLCS',  'DACHS', 'QUASAR',  'YCR-BCIP', 'MECC', 'Epi700',  'TCGA',   'MUNICH','DUSSEL',  'CPTAC',]
data_size = [2068,   2039,    1774,     867,         683,    603,       426,      287,  196, 105,                                                       ]

In [ ]:
# sorted by size of the dataset + HistAuGAN augmentation
histaugan = [
    [0.96, 0.91, 0.92, 0.89, 0.84, 0.91, 0.84, 0.81, 0.70, 0.78], # 0.88],
    [0.92, 0.98, 0.91, 0.84, 0.81, 0.89, 0.84, 0.78, 0.74, 0.80], # 0.88],
    [0.93, 0.92, 0.96, 0.90, 0.84, 0.92, 0.86, 0.83, 0.74, 0.77], # 0.89],
    [0.94, 0.93, 0.95, 0.95, 0.87, 0.94, 0.88, 0.84, 0.77, 0.85], # 0.92],
    [0.77, 0.76, 0.76, 0.77, 0.72, 0.76, 0.73, 0.69, 0.64, 0.71], # 0.74],
    [0.93, 0.91, 0.93, 0.90, 0.85, 0.92, 0.86, 0.83, 0.71, 0.81], # 0.90],
    [0.88, 0.89, 0.89, 0.86, 0.79, 0.88, 0.90, 0.79, 0.67, 0.74], # 0.87],
    [0.85, 0.84, 0.88, 0.84, 0.79, 0.86, 0.82, 0.83, 0.68, 0.72], # 0.79],
    [0.81, 0.82, 0.81, 0.83, 0.80, 0.79, 0.75, 0.76, 0.71, 0.74], # 0.78],
    [0.89, 0.87, 0.91, 0.88, 0.81, 0.87, 0.85, 0.82, 0.64, 0.84], # 0.89],
    # [0.90, 0.88, 0.88, 0.83, 0.71, 0.88, 0.83, 0.67, 0.61, 0.67, 0.96],
]

histaugan = np.array(histaugan)
cohorts = ['NLCS',  'DACHS', 'QUASAR',  'YCR-BCIP', 'MECC', 'Epi700',  'TCGA',   'MUNICH','DUSSEL',  'CPTAC',]
data_size = [2068,   2039,    1774,     867,         683,    603,       426,      287,  196, 105,                                                       ]

In [ ]:
# sorted by size of the dataset 
raw = [
    [0.93, 0.91, 0.91, 0.90, 0.85, 0.89, 0.82, 0.77, 0.79, 0.78],
    [0.88, 0.96, 0.90, 0.85, 0.82, 0.87, 0.89, 0.77, 0.79, 0.76],
    [0.92, 0.92, 0.94, 0.91, 0.84, 0.92, 0.86, 0.79, 0.82, 0.75],
    [0.95, 0.94, 0.95, 0.95, 0.87, 0.94, 0.90, 0.82, 0.86, 0.82],
    [0.78, 0.75, 0.76, 0.76, 0.74, 0.76, 0.73, 0.65, 0.71, 0.68],
    [0.91, 0.91, 0.93, 0.91, 0.85, 0.93, 0.87, 0.81, 0.80, 0.80],
    [0.87, 0.88, 0.89, 0.86, 0.79, 0.86, 0.87, 0.78, 0.77, 0.73],
    [0.85, 0.86, 0.89, 0.86, 0.79, 0.88, 0.86, 0.81, 0.76, 0.70],
    [0.81, 0.82, 0.81, 0.82, 0.79, 0.78, 0.76, 0.71, 0.73, 0.72],
    [0.92, 0.88, 0.86, 0.91, 0.79, 0.81, 0.82, 0.75, 0.77, 0.83],
]

raw = np.array(raw)
cohorts = ['NLCS',  'DACHS', 'QUASAR',  'YCR-BCIP', 'MECC', 'Epi700',  'TCGA',   'MUNICH','DUSSEL',  'CPTAC',]
data_size = [2068,   2039,    1774,      867,        683,    603,       426,      287,     196,       105,                                                       ]

In [ ]:
diff = histaugan - ours

In [ ]:
plot = macenko
name = 'macenko'
target_label = 'MSI-H'

fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(plot.T, vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(cohorts)), labels=cohorts)
ax.set_yticks(np.arange(len(cohorts)), labels=cohorts)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
# for i in range(len(cohorts)):
#     for j in range(len(cohorts)):
#         text = ax.text(j, i, plot[i][j],
#                        ha="center", va="center", color="w")

# ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()

ax.set(
    # title=f"AUROCs for single cohort experiments (target: {target_label})",
    title=f"AUROCs for single cohort experiments with {name}",
)

# if not transposed:
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
# else:
ax.set_ylabel('Train')
ax.set_xlabel('Test')

cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
# fig.savefig(Path('figures') / f'results_{name}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
plot = macenko
name = 'macenko'

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=((6/5)*5, 5), gridspec_kw={'width_ratios': [5, 1], 'wspace':0, 'hspace':0}) # figsize=(7, 5) 'width_ratios': [6, 1]
im = ax[0].imshow(plot.T, vmin=0.55, vmax=1, cmap='plasma')


ax[0].set_xticks(np.arange(len(cohorts)), labels=cohorts)
ax[0].set_yticks(np.arange(len(cohorts)), labels=cohorts)

plt.setp(ax[0].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()

ax[0].set(
    title=f"AUROC scores (target: {target_label})",
)

# if not transposed:
# ax[0].set_xlabel('Train')
# ax[0].set_ylabel('Test')
# else:
ax[0].set_ylabel('Train')
ax[0].set_xlabel('Test')

# plot bar plot
# ax[1].set_box_aspect(5/1)
ax[1].barh(range(len(data_size), 0, -1), data_size, color='slategray')

ax[1].axes.get_yaxis().set_visible(False)
ax[1].set_xticks((0, 1000, 2000))
plt.setp(ax[1].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
# ax[1].spines['bottom'].set_visible(False)
# ax[1].spines['left'].set_visible(False)
ax[1].set(
    ylim=[0.5, len(data_size) + 0.5],
    # ylim=[-0.05, 1.05],
)
ax[1].set_xlabel('Counts')


# plt.title("AUROC values (target: MSI)")
# cax = fig.add_axes([ax[0].get_position().x1+0.01,ax[0].get_position().y0,0.02,ax[0].get_position().height])
# plt.colorbar(im, cax=cax)
# fig.subplots_adjust(wspace=0, hspace=0)
# plt.tight_layout()
fig.savefig(Path('figures') / f'results_heatmap_{name}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
diff = histaugan - macenko
name = 'histaugan-macenko'
target_label = 'MSI-H'

fig, ax = plt.subplots(figsize=(5, 5))
# im = ax.imshow(diff.T, vmin=-np.abs(diff).max(), vmax=np.abs(diff).max(), cmap='RdBu') # vmin=0.55, vmax=1
im = ax.imshow(diff.T, vmin=-0.1, vmax=0.1, cmap='RdBu') # vmin=0.55, vmax=1


ax.set_xticks(np.arange(len(cohorts)), labels=cohorts)
ax.set_yticks(np.arange(len(cohorts)), labels=cohorts)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
# for i in range(len(cohorts)):
#     for j in range(len(cohorts)):
#         text = ax.text(j, i, ours[i][j],
#                        ha="center", va="center", color="w")

# ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()

ax.set(
    title=f"Differences in AUROC scores in single cohort experiments",
)

# if not transposed:
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
# else:
ax.set_ylabel('Train')
ax.set_xlabel('Test')

cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
# cax.set_yticklabels(['No aug. better', 'HistAuGAN better'])
# fig.savefig(Path('figures') / f'results_heatmap_diff_{name}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Heatmaps for MSI (other methods)

In [ ]:
attmil = [
    [0.79, 0.82, 0.84, 0.80, 0.77, 0.66, 0.68, 0.73, 0.77, 0.78, 0.83],
    [0.77, 0.91, 0.91, 0.88, 0.85, 0.71, 0.75, 0.80, 0.76, 0.80, 0.89],
    [0.75, 0.84, 0.93, 0.87, 0.82, 0.70, 0.71, 0.75, 0.75, 0.78, 0.86],
    [0.73, 0.85, 0.91, 0.91, 0.88, 0.72, 0.73, 0.77, 0.76, 0.79, 0.88],
    [0.74, 0.87, 0.90, 0.89, 0.90, 0.76, 0.73, 0.71, 0.76, 0.84, 0.89],
    [0.66, 0.81, 0.86, 0.83, 0.86, 0.71, 0.68, 0.68, 0.65, 0.81, 0.83],
    [0.64, 0.74, 0.77, 0.73, 0.68, 0.64, 0.69, 0.76, 0.71, 0.65, 0.77],
    [0.66, 0.73, 0.75, 0.76, 0.73, 0.65, 0.66, 0.66, 0.65, 0.68, 0.74],
    [0.74, 0.83, 0.85, 0.82, 0.84, 0.64, 0.74, 0.69, 0.82, 0.70, 0.77],
    [0.73, 0.80, 0.84, 0.81, 0.74, 0.69, 0.65, 0.58, 0.68, 0.92, 0.83],
    [0.82, 0.90, 0.90, 0.92, 0.84, 0.78, 0.79, 0.80, 0.80, 0.86, 0.91],
]
attmil = np.array(attmil)

In [ ]:
# sorted by cohort size
attmil = [
    [0.91, 0.91, 0.85, 0.88, 0.77, 0.88, 0.73, 0.76, 0.73, 0.72],
    [0.87, 0.93, 0.84, 0.86, 0.75, 0.82, 0.75, 0.75, 0.71, 0.70],
    [0.88, 0.91, 0.91, 0.89, 0.80, 0.85, 0.77, 0.76, 0.75, 0.71],
    [0.92, 0.90, 0.90, 0.91, 0.80, 0.84, 0.82, 0.80, 0.79, 0.78],
    [0.76, 0.75, 0.73, 0.74, 0.66, 0.73, 0.66, 0.65, 0.66, 0.65],
    [0.89, 0.90, 0.87, 0.89, 0.71, 0.90, 0.74, 0.76, 0.73, 0.76],
    [0.80, 0.84, 0.82, 0.83, 0.73, 0.77, 0.79, 0.77, 0.68, 0.66],
    [0.82, 0.85, 0.83, 0.77, 0.69, 0.84, 0.74, 0.82, 0.74, 0.64],
    [0.73, 0.77, 0.74, 0.77, 0.76, 0.68, 0.64, 0.71, 0.69, 0.64],
    [0.83, 0.86, 0.81, 0.83, 0.68, 0.86, 0.66, 0.65, 0.68, 0.71],
]
attmil = np.array(attmil)

In [ ]:
attmil = np.delete(np.delete(attmil, 9, axis=0), 9, axis=1)

In [ ]:
attmil

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(attmil, vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(cohorts)), labels=cohorts)
ax.set_yticks(np.arange(len(cohorts)), labels=cohorts)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
# for i in range(len(cohorts)):
#     for j in range(len(cohorts)):
#         text = ax.text(j, i, ours[i][j],
#                        ha="center", va="center", color="w")

# ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()

ax.set(
    title="AUROCs for single cohort experiments (target: MSI)",
)

ax.set_xlabel('Train')
ax.set_ylabel('Test')

cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
# fig.savefig(Path('figures') / f'results_heatmap_attmil.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
# cohorts_small = ['TCGA', 'QUASAR', 'DACHS', 'NLCS']
cohorts_small = ['NLCS', 'DACHS', 'QUASAR', 'TCGA']

fig, ax = plt.subplots(figsize=((3/4)*2.5, 2.5))
im = ax.imshow(np.flip(echles.T, axis=(0,1)), vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(cohorts_small)), labels=cohorts_small)
ax.set_yticks(np.arange(len(cohorts_small)), labels=cohorts_small)

labels = np.flip(echles.T)
# Loop over data dimensions and create text annotations.
for i in range(len(cohorts_small)):
    for j in range(len(cohorts_small)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=10)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# ax.set(
#     title="CNN",
# )

# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
ax.set_xlabel('Test')
ax.set_ylabel('CNN')

# fig.savefig(Path('figures') / f'results_cnn.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
ours = [
    [0.86, 0.87, 0.87, 0.85, 0.83, 0.75, 0.68, 0.80, 0.81, 0.86, 0.87],
    [0.87, 0.92, 0.92, 0.92, 0.91, 0.75, 0.77, 0.84, 0.84, 0.86, 0.92],
    [0.86, 0.89, 0.96, 0.89, 0.88, 0.75, 0.73, 0.83, 0.82, 0.86, 0.89],
    [0.85, 0.91, 0.92, 0.93, 0.91, 0.78, 0.72, 0.84, 0.83, 0.88, 0.91],
    [0.85, 0.91, 0.92, 0.90, 0.92, 0.81, 0.74, 0.81, 0.84, 0.89, 0.91],
    [0.82, 0.89, 0.85, 0.90, 0.84, 0.86, 0.67, 0.72, 0.71, 0.88, 0.91],
    [0.76, 0.76, 0.81, 0.76, 0.75, 0.72, 0.75, 0.75, 0.79, 0.78, 0.79],
    [0.75, 0.74, 0.75, 0.75, 0.75, 0.67, 0.66, 0.73, 0.85, 0.73, 0.73],
    [0.86, 0.85, 0.86, 0.81, 0.88, 0.73, 0.71, 0.79, 0.73, 0.80, 0.85],
    [0.80, 0.84, 0.87, 0.86, 0.84, 0.73, 0.68, 0.62, 0.69, 0.94, 0.88],
    [0.91, 0.94, 0.93, 0.94, 0.92, 0.83, 0.82, 0.86, 0.87, 0.92, 0.95],
]
ours = np.array(ours)

In [ ]:
# macenko (8 epochs)
# training cohorts: NLCS	Dachs	Quasar	TCGA
macenko_zoom = [
    [0.93, 0.92, 0.92, 0.83],
    [0.92, 0.96, 0.92, 0.84],
    [0.93, 0.92, 0.95, 0.85],
    [0.88, 0.87, 0.88, 0.86],   
]
macenko_zoom = np.array(macenko_zoom)

In [ ]:
# HistAuGAN augmentation
# training cohorts: NLCS	Dachs	Quasar	TCGA
histaugan = [
    [0.96, 0.91, 0.92, 0.84,],
    [0.92, 0.98, 0.91, 0.84,],
    [0.93, 0.92, 0.96, 0.86,],
    [0.88, 0.89, 0.89, 0.90,],
]

histaugan = np.array(histaugan)
# cohorts = ['NLCS',  'DACHS', 'QUASAR',  'YCR-BCIP', 'MECC', 'Epi700',  'TCGA',   'MUNICH','DUSSEL',  'CPTAC',]
# data_size = [2068,   2039,    1774,     867,         683,    603,       426,      287,  196, 105,                                                       ]

In [ ]:
histaugan.T

In [ ]:
cohorts_small = ['NLCS', 'DACHS', 'QUASAR', 'TCGA']
plot = macenko_zoom
name = 'macenko'

fig, ax = plt.subplots(figsize=(1.875, 1.875))
# im = ax.imshow(np.flip(ours[:4, :4].T), vmin=0.55, vmax=1, cmap='plasma')
im = ax.imshow(plot.T, vmin=0.55, vmax=1, cmap='plasma')


ax.set_xticks(np.arange(len(cohorts_small)), labels=cohorts_small)
ax.axes.get_xaxis().set_visible(False)

ax.set_yticks(np.arange(len(cohorts_small)), labels=cohorts_small)

labels = plot.T
# Loop over data dimensions and create text annotations.
for i in range(len(cohorts_small)):
    for j in range(len(cohorts_small)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=10)
        
# ax.set(
#     title="Transformer",
# )

# ax.set_xlabel('Train')
ax.set_ylabel('Transformer')

fig.savefig(Path('figures') / f'results_transformer_{name}_zoom.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()

In [ ]:
# cohorts_small = ['TCGA', 'QUASAR', 'DACHS', 'NLCS']

fig, ax = plt.subplots(figsize=(1.875, 1.875))
im = ax.imshow(np.flip(attmil[:4, :4].T), vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(cohorts_small)), labels=cohorts_small)
ax.axes.get_xaxis().set_visible(False)

ax.set_yticks(np.arange(len(cohorts_small)), labels=cohorts_small)

labels = np.flip(attmil[:4, :4].T)
# Loop over data dimensions and create text annotations.
for i in range(len(cohorts_small)):
    for j in range(len(cohorts_small)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=10)
        
# ax.set(
#     title="AttentionMIL",
# )

# ax.set_xlabel('Train')
ax.set_ylabel('AttentionMIL')

# fig.savefig(Path('figures') / f'results_attmil_zoom.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()

## Heatmaps for BRAF

In [ ]:
braf_cohorts = ['TCGA', 'Quasar', 'Dachs', 'NLCS', 'Belfast']
# BRAF cohorts ordered by size
braf_cohorts = ['Dachs', 'Rainbow', 'Quasar', 'Belfast', 'TCGA']
braf_cohorts_labels = ['DACHS', 'NLCS', 'QUASAR', 'Epi700', 'TCGA']
cohort_size, target = [2075, 2038, 1477, 641, 500], "BRAF"

In [ ]:
braf = np.zeros((5, 5))
for i, c_i in enumerate(braf_cohorts):
    for j, c_j in enumerate(braf_cohorts):
        braf[i, j] = braf_kras[braf_kras["Target"] == "braf"][braf_kras["Train"]==c_j][braf_kras["Test"]==c_i]["auroc mean"].item()

In [ ]:
braf_histaugan = np.zeros((5, 5))
for i, c_i in enumerate(braf_cohorts):
    for j, c_j in enumerate(braf_cohorts):
        braf_histaugan[i, j] = braf_kras_histaugan[braf_kras_histaugan["Target"] == "braf"][braf_kras_histaugan["Train"]==c_j][braf_kras_histaugan["Test"]==c_i]["auroc mean"].item()

In [ ]:
braf_kras[braf_kras["Target"] == "braf"][braf_kras["Train"]==c_j][braf_kras["Test"]==c_i]["auroc mean"].item()

In [ ]:
diff = braf_histaugan - braf

In [ ]:
diff

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(braf.T, vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)
ax.set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)

labels = braf.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)
ax.set(
    title="AUROC scores (target: BRAF)",
)
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
ax.set_xlabel('Test')
ax.set_ylabel('Train')
# cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
# plt.colorbar(im, cax=cax)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# fig.savefig(Path('figures') / f'results_{target}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=((6/5)*5, 5), gridspec_kw={'width_ratios': [5, 1], 'wspace':0, 'hspace':0}) # figsize=(7, 5) 'width_ratios': [6, 1]
im = ax[0].imshow(braf.T, vmin=0.55, vmax=1, cmap='plasma')


ax[0].set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)
ax[0].set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)

plt.setp(ax[0].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
fig.tight_layout()

ax[0].set(
    title="AUROC scores (target: BRAF)",
)

labels = braf.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax[0].text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)

# if not transposed:
# ax[0].set_xlabel('Train')
# ax[0].set_ylabel('Test')
# else:
ax[0].set_ylabel('Train')
ax[0].set_xlabel('Test')

# plot bar plot
# ax[1].set_box_aspect(5/1)
ax[1].barh(range(len(cohort_size), 0, -1), cohort_size, color='slategray')

ax[1].axes.get_yaxis().set_visible(False)
ax[1].set_xticks((0, 1000, 2000))
plt.setp(ax[1].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
# ax[1].spines['bottom'].set_visible(False)
# ax[1].spines['left'].set_visible(False)
ax[1].set(
    ylim=[0.5, len(cohort_size) + 0.5],
    # ylim=[-0.05, 1.05],
)
ax[1].set_xlabel('Counts')


# plt.title("AUROC values (target: MSI)")
# cax = fig.add_axes([ax[0].get_position().x1+0.01,ax[0].get_position().y0,0.02,ax[0].get_position().height])
# plt.colorbar(im, cax=cax)
fig.subplots_adjust(wspace=0, hspace=0)
# plt.tight_layout()
# fig.savefig(Path('figures') / f'results_heatmap_{target}_histaugan.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(diff.T, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')

ax.set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)
ax.set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)

labels = diff.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)
ax.set(
    title="AUROC scores (target: BRAF)",
)
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
ax.set_xlabel('Test')
ax.set_ylabel('Train')
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# fig.savefig(Path('figures') / f'diff_{target}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:
braf

In [ ]:
kras = np.zeros((5, 5))
for i, c_i in enumerate(braf_cohorts):
    for j, c_j in enumerate(braf_cohorts):
        kras[i, j] = braf_kras[braf_kras["Target"] == "kras"][braf_kras["Train"]==c_j][braf_kras["Test"]==c_i]["auroc mean"].item()

In [ ]:
kras_histaugan = np.zeros((5, 5))
for i, c_i in enumerate(braf_cohorts):
    for j, c_j in enumerate(braf_cohorts):
        kras_histaugan[i, j] = braf_kras_histaugan[braf_kras_histaugan["Target"] == "kras"][braf_kras_histaugan["Train"]==c_j][braf_kras_histaugan["Test"]==c_i]["auroc mean"].item()

In [ ]:
cohort_size, target = [2068, 2033, 1436, 645, 500], "KRAS"

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(kras_histaugan.T, vmin=0.55, vmax=1, cmap='plasma')

ax.set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts)
ax.set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts)

labels = kras_histaugan.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)
ax.set(
    title=f"AUROC scores (target: {target})",
)
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
ax.set_xlabel('Test')
ax.set_ylabel('Train')
# cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
# plt.colorbar(im, cax=cax)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# fig.savefig(Path('figures') / f'results_{target}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=((6/5)*5, 5), gridspec_kw={'width_ratios': [5, 1], 'wspace':0, 'hspace':0}) # figsize=(7, 5) 'width_ratios': [6, 1]
im = ax[0].imshow(kras_histaugan.T, vmin=0.55, vmax=1, cmap='plasma')


ax[0].set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)
ax[0].set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)

plt.setp(ax[0].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
fig.tight_layout()

ax[0].set(
    title=f"AUROC scores (target: {target})",
)

labels = kras_histaugan.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax[0].text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)

# if not transposed:
# ax[0].set_xlabel('Train')
# ax[0].set_ylabel('Test')
# else:
ax[0].set_ylabel('Train')
ax[0].set_xlabel('Test')

# plot bar plot
# ax[1].set_box_aspect(5/1)
ax[1].barh(range(len(cohort_size), 0, -1), cohort_size, color='slategray')

ax[1].axes.get_yaxis().set_visible(False)
ax[1].set_xticks((0, 1000, 2000))
plt.setp(ax[1].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
# ax[1].spines['bottom'].set_visible(False)
# ax[1].spines['left'].set_visible(False)
ax[1].set(
    ylim=[0.5, len(cohort_size) + 0.5],
    # ylim=[-0.05, 1.05],
)
ax[1].set_xlabel('Counts')


# plt.title("AUROC values (target: MSI)")
# cax = fig.add_axes([ax[0].get_position().x1+0.01,ax[0].get_position().y0,0.02,ax[0].get_position().height])
# plt.colorbar(im, cax=cax)
fig.subplots_adjust(wspace=0, hspace=0)
# plt.tight_layout()
# fig.savefig(Path('figures') / f'results_heatmap_{target}_histaugan.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
diff = kras_histaugan - kras

In [ ]:
diff

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(diff.T, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')

ax.set_xticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)
ax.set_yticks(np.arange(len(braf_cohorts)), labels=braf_cohorts_labels)

labels = diff.T
# Loop over data dimensions and create text annotations.
for i in range(len(braf_cohorts)):
    for j in range(len(braf_cohorts)):
        text = ax.text(j, i, f'{labels[i][j]:.2f}',
                       ha="center", va="center", color="w", fontsize=14)
ax.set(
    title="AUROC scores (target: KRAS)",
)
# ax.set_xlabel('Train')
# ax.set_ylabel('Test')
ax.set_xlabel('Test')
ax.set_ylabel('Train')
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
        
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# fig.savefig(Path('figures') / f'diff_{target}.svg', format='svg', bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:
def heatmap(x, y, size):
    fig, ax = plt.subplots()
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 500
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size * size_scale, # Vector of square sizes, proportional to size parameter
        marker='o' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/drazenz/heatmap/master/autos.clean.csv')
columns = ['bore', 'stroke', 'compression-ratio', 'horsepower', 'city-mpg', 'price'] 
corr = data[columns].corr()
corr = pd.melt(corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
corr.columns = ['x', 'y', 'value']
heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value'].abs()
)

In [ ]:
corr

# Donut plots for cohort overview

In [ ]:
# all cohorts
cohorts_all = ['CPTAC', 'DACHS', 'DUSSEL', 'ERLANGEN', 'Epi700', 'FOXTROT', 'CHINA', 'MAINZ', 'MCO', 'MECC', 'MUNICH', 'RAINBOW', 'QUASAR', 'TCGA', 'TRANSCOT', 'YCR-BCIP-resections']

In [ ]:
# MSI cohorts
# target = 'MSI'
# cohorts = ['CPTAC', 'DACHS', 'DUSSEL', 'ERLANGEN', 'Epi700', 'FOXTROT', 'CHINA', 'MAINZ', 'MCO', 'MECC', 'MUNICH', 'RAINBOW', 'QUASAR', 'TCGA', 'TRANSCOT', 'YCR-BCIP-resections']
# target = 'BRAF'
target = 'KRAS'
cohorts = ['DACHS',  'Epi700', 'MCO',  'QUASAR', 'RAINBOW', 'TCGA', ]
# cohorts = ['CPTAC', 'DACHS', 'DUSSEL', 'Epi700', 'MECC', 'MUNICH', 'NLCS', 'QUASAR', 'TCGA', 'YCR-BCIP']
# data, target = [105, 2039, 196, 603, 683, 287,  2068, 1774, 426, 867], "MSI"
# BRAF cohorts
# cohorts = ['DACHS', 'Epi700', 'NLCS', 'QUASAR', 'TCGA']
# cohort_ids = [1, 3, 6, 7, 8]
# data, target = [2075, 641, 2038, 1477,  500], "BRAF"
# KRAS cohorts
# data, target = [2068, 645, 2033, 1436,  500], "KRAS"

In [ ]:
# MSI cohorts
# cohort_size = {
#     'CHINA': 35,
#     'CPTAC': 105,
#     'DACHS': 2039,
#     'DUSSEL': 196,
#     'Epi700': 603, 
#     'ERLANGEN': 458,
#     'FOXTROT': 702,
#     'MAINZ': 86,
#     'MCO': 1388, 
#     'MECC': 683, 
#     'MUNICH': 287, 
#     'QUASAR': 1774,
#     'RAINBOW': 2068, 
#     'TCGA': 426, 
#     'TRANSCOT': 1972, 
#     'YCR-BCIP-resections': 867
# }

# BRAF cohorts
cohort_size = {
    'DACHS': 2075,
    'Epi700': 641, 
    'MCO': 1388, 
    'QUASAR': 1477,
    'RAINBOW': 2038, 
    'TCGA': 500, 
}

# KRAS cohorts
# cohort_size = {
#     'DACHS': 2068,
#     'Epi700': 645, 
#     'MCO': 1390, 
#     'QUASAR': 1436,
#     'RAINBOW': 2033, 
#     'TCGA': 500, 
# }

In [ ]:
sum(cohort_size.values())

In [ ]:
cohorts_ordered = sorted(cohorts, key=lambda x: cohort_label[x])
size_ordered = [cohort_size[c] for c in cohorts_ordered]
label_ordered = [cohort_label[c] for c in cohorts_ordered]
cohort_ids = [cohorts_all.index(c) for c in cohorts_ordered]
print(f'{len(cohorts)} cohorts with {sum(cohort_size.values())} patients')
print(label_ordered)
print(size_ordered)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(aspect="equal"))

cmap = plt.get_cmap("twilight")
r = np.array(cohort_ids) / len(cohorts_all)
colors = cmap(r)

wedges, texts = ax.pie(size_ordered, wedgeprops=dict(width=0.5, linewidth=2, edgecolor='w'), startangle=90, colors=colors)

plt.legend(label_ordered, bbox_to_anchor=(1.35, 0.5), loc='right', fontsize=16)

plt.savefig(figure_path / f"num_patients_{target}.svg", format='svg', bbox_inches = 'tight', pad_inches = 0)
plt.show()

In [ ]:
print(r)

In [ ]:
perm